In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import torch
import torch.nn as nn
import albumentations as A
import timm
import numpy as np
import pandas as pd
import random
import os
import json
import cv2

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, Dataset

In [10]:
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [40]:
epoch = 30
num_workers = 2
batch_size = 48
lr = 1e-4
early_stop = 5
k_fold_num = 5
model_name_ = "swsl_resnext50_32x4d"

In [38]:
def train(data_loader):
    model = timm.create_model(model_name_, pretrained=True, num_classes=7).to(device)
    class_num = [329, 205, 235, 134, 151, 245, 399]
    class_weight = torch.tensor(np.max(class_num)/class_num).to(device, dtype=torch.float)
    criterion = nn.CrossEntropyLoss(weight=class_weight)

    feat_extractor = [m for n, m in model.named_parameters() if "fc" not in n]
    classifier = [p for p in model.fc.parameters()]
    params = [
        {"params": feat_extractor, "lr": lr*0.5},
         {"params": classifier, "lr": lr}
    ]
    optimizer = Adam(params, lr=lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

    result = {"train_loss": [], "val_loss": [], "val_acc": [], "val_f1": []}

    train_loader = data_loader["train_loader"]
    val_loader = data_loader["val_loader"]

    for epoch_idx in range(1, epoch + 1):
        model.train()

        iter_train_loss = []
        iter_val_loss = []
        iter_val_acc = []
        iter_val_f1 = []

    for iter_idx, (train_imgs, train_labels) in enumerate(train_loader, 1):
        train_imgs, train_labels = train_imgs.to(device, dtype=torch.float), train_labels.to(device)

        optimizer.zero_grad()

        train_pred = model(train_imgs)
        train_loss = criterion(train_pred, train_labels)
        train_loss.backward()

        optimizer.step()
        iter_train_loss.append(train_loss.cpu().item())

        print(f"[Epoch {epoch_idx}/{epoch}] model training iteration {iter_idx}/{len(train_loader)}     ",
                    end="\r")

        with torch.no_grad():
            for iter_idx, (val_imgs, val_labels) in enumerate(val_loader, 1):
                model.eval()

                val_imgs, val_labels = val_imgs.to(device, dtype=torch.float), val_labels.to(device)

                val_pred = model(val_imgs)
                val_loss = criterion(val_pred, val_labels)

                iter_val_loss.append(val_loss.cpu().item())

                val_pred_c = val_pred.argmax(dim=1)
                iter_val_acc.extend((val_pred_c == val_labels).cpu().tolist())

                iter_f1_score = f1_score(y_true=val_labels.cpu().numpy(), y_pred=val_pred_c.cpu().numpy(), average="macro")
                iter_val_f1.append(iter_f1_score)

                print(
                  f"[Epoch {epoch_idx}/{epoch}] model validation iteration {iter_idx}/{len(val_loader)}     ",
                    end="\r"
                )
        epoch_train_loss = np.mean(iter_train_loss)
        epoch_val_loss = np.mean(iter_val_loss)
        epoch_val_acc = np.mean(iter_val_acc)
        epoch_val_f1 = np.mean(iter_val_f1)

        result["train_loss"].append(epoch_train_loss)
        result["val_loss"].append(epoch_val_loss)
        result["val_acc"].append(epoch_val_acc)
        result["val_f1"].append(epoch_val_f1)

        scheduler.step()

        print(
            f"[Epoch {epoch_idx}/{epoch}] "
            f"train loss : {epoch_train_loss:.4f} | "
            f"valid loss : {epoch_val_loss:.4f} | valid acc : {epoch_val_acc:.2f}% | valid f1 score : {epoch_val_f1:.4f}"
        )

        Best_Model = None
        Best_f1 = 0
        stop_count = 0

        if epoch_val_f1 > Best_f1:
            Best_f1 = epoch_val_f1
            Best_Model = model.state_dict()
            stop_count = 0
        else:
            stop_count += 1

        if stop_count == early_stop:
            print("early stoped." + " " * 30)
            break

    return result, Best_Model

In [33]:
class_encoder = {
    'dog': 0,
    'elephant': 1,
    'giraffe': 2,
    'guitar': 3,
    'horse': 4,
    'house': 5,
    'person': 6
}

def img_gather(img_path):
    class_list = os.listdir(img_path)

    file_lists = []
    label_lists = []

    for class_name in class_list:
        file_list = os.listdir(os.path.join(img_path, class_name))
        file_list = list(map(lambda x: "/".join([img_path] + [class_name] + [x]), file_list))
        label_list = [class_encoder[class_name]] * len(file_list)

        file_lists.extend(file_list)
        label_lists.extend(label_list)

    file_lists = np.array(file_lists)
    label_lists = np.array(label_lists)

    return file_lists, label_lists


class TrainDataset(Dataset):
    def __init__(self, file_lists, label_lists, transforms=None):
        self.file_lists = file_lists.copy()
        self.label_lists = label_lists.copy()
        self.transforms = transforms

    def __getitem__(self, idx):
        img = cv2.imread(self.file_lists[idx], cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            img = self.transforms(image=img)["image"]

        img = img.transpose(2, 0, 1)

        label = self.label_lists[idx]

        img = torch.tensor(img, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.long)

        return img, label

    def __len__(self):
        assert len(self.file_lists) == len(self.label_lists)
        return len(self.file_lists)


class TestDataset(Dataset):
    def __init__(self, file_lists, transforms=None):
        self.file_lists = file_lists.copy()
        self.transforms = transforms

    def __getitem__(self, idx):
        img = cv2.imread(self.file_lists[idx], cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            img = self.transforms(image=img)["image"]

        img = img.transpose(2, 0, 1)

        img = torch.tensor(img, dtype=torch.float)

        return img

    def __len__(self):
        return len(self.file_lists)

In [42]:
train_transforms = A.Compose([
    A.Rotate(),
    A.HorizontalFlip(),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    A.Normalize()
])

valid_transforms = A.Compose([
    A.Normalize()
])
data_lists, data_labels = img_gather("/content/drive/MyDrive/train")

best_models = []
k_fold_num = k_fold_num

if k_fold_num == -1:
    train_lists, valid_lists, train_labels, valid_labels = train_test_split(data_lists, data_labels, train_size=0.8,
                                                                            shuffle=True, random_state=random_seed,
                                                                            stratify=data_labels)

    train_dataset = TrainDataset(file_lists=train_lists, label_lists=train_labels, transforms=train_transforms)
    valid_dataset = TrainDataset(file_lists=valid_lists, label_lists=valid_labels, transforms=valid_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

    data_loader = {"train_loader": train_loader, "valid_loader": valid_loader}

    print("No fold training starts ... ")
    train_result, best_model = train(data_loader)

    best_models.append(best_model)

else:
    skf = StratifiedKFold(n_splits=k_fold_num, random_state=random_seed, shuffle=True)

    print(f"{k_fold_num} fold training starts ... ")
    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(data_lists, data_labels), 1):
        print(f"- {fold_idx} fold -")
        train_lists, train_labels = data_lists[train_idx], data_labels[train_idx]
        val_lists, valid_labels = data_lists[valid_idx], data_labels[valid_idx]

        train_dataset = TrainDataset(file_lists=train_lists, label_lists=train_labels, transforms=train_transforms)
        val_dataset = TrainDataset(file_lists=val_lists, label_lists=valid_labels, transforms=valid_transforms)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

        data_loader = {"train_loader": train_loader, "val_loader": val_loader}

        train_result, best_model = train(data_loader)

        best_models.append(best_model)

        model_save_path = f"best_model_fold_{fold_idx}.pt"
        torch.save(best_model, model_save_path)
        print(f"Saved model for fold {fold_idx} to {model_save_path}")

5 fold training starts ... 
- 1 fold -


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


[Epoch 30/30] train loss : 2.0016 | valid loss : 1.8884 | valid acc : 0.20% | valid f1 score : 0.1149
[Epoch 30/30] train loss : 1.9265 | valid loss : 1.8251 | valid acc : 0.25% | valid f1 score : 0.1644
[Epoch 30/30] train loss : 1.8860 | valid loss : 1.7251 | valid acc : 0.36% | valid f1 score : 0.2813
[Epoch 30/30] train loss : 1.8106 | valid loss : 1.6111 | valid acc : 0.46% | valid f1 score : 0.3631
[Epoch 30/30] train loss : 1.7158 | valid loss : 1.4854 | valid acc : 0.54% | valid f1 score : 0.4662
[Epoch 30/30] train loss : 1.6067 | valid loss : 1.3774 | valid acc : 0.60% | valid f1 score : 0.5325
[Epoch 30/30] train loss : 1.5204 | valid loss : 1.2807 | valid acc : 0.64% | valid f1 score : 0.5797
[Epoch 30/30] train loss : 1.4240 | valid loss : 1.1932 | valid acc : 0.67% | valid f1 score : 0.6116
[Epoch 30/30] train loss : 1.3563 | valid loss : 1.1230 | valid acc : 0.69% | valid f1 score : 0.6371
[Epoch 30/30] train loss : 1.2753 | valid loss : 1.0618 | valid acc : 0.71% | vali

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


[Epoch 30/30] train loss : 1.9346 | valid loss : 1.8421 | valid acc : 0.30% | valid f1 score : 0.1773
[Epoch 30/30] train loss : 1.8767 | valid loss : 1.8184 | valid acc : 0.34% | valid f1 score : 0.2201
[Epoch 30/30] train loss : 1.8979 | valid loss : 1.7207 | valid acc : 0.41% | valid f1 score : 0.2812
[Epoch 30/30] train loss : 1.7925 | valid loss : 1.6574 | valid acc : 0.45% | valid f1 score : 0.3369
[Epoch 30/30] train loss : 1.7705 | valid loss : 1.5876 | valid acc : 0.48% | valid f1 score : 0.3735
[Epoch 30/30] train loss : 1.6749 | valid loss : 1.5106 | valid acc : 0.51% | valid f1 score : 0.4170
[Epoch 30/30] train loss : 1.6014 | valid loss : 1.4409 | valid acc : 0.53% | valid f1 score : 0.4498
[Epoch 30/30] train loss : 1.5447 | valid loss : 1.3787 | valid acc : 0.56% | valid f1 score : 0.4768
[Epoch 30/30] train loss : 1.4760 | valid loss : 1.3300 | valid acc : 0.58% | valid f1 score : 0.5004
[Epoch 30/30] train loss : 1.4369 | valid loss : 1.2802 | valid acc : 0.60% | vali

...

[Epoch 30/30] train loss : 1.9226 | valid loss : 1.8568 | valid acc : 0.29% | valid f1 score : 0.2285
[Epoch 30/30] train loss : 1.8632 | valid loss : 1.7719 | valid acc : 0.34% | valid f1 score : 0.2701
[Epoch 30/30] train loss : 1.8754 | valid loss : 1.6930 | valid acc : 0.40% | valid f1 score : 0.3135
[Epoch 30/30] train loss : 1.8168 | valid loss : 1.6038 | valid acc : 0.46% | valid f1 score : 0.3890
[Epoch 30/30] train loss : 1.7633 | valid loss : 1.5200 | valid acc : 0.51% | valid f1 score : 0.4444
[Epoch 30/30] train loss : 1.6994 | valid loss : 1.4259 | valid acc : 0.57% | valid f1 score : 0.5136
[Epoch 30/30] train loss : 1.6076 | valid loss : 1.3443 | valid acc : 0.61% | valid f1 score : 0.5657
[Epoch 30/30] train loss : 1.5485 | valid loss : 1.2693 | valid acc : 0.65% | valid f1 score : 0.6075
[Epoch 30/30] train loss : 1.4848 | valid loss : 1.2111 | valid acc : 0.68% | valid f1 score : 0.6383
[Epoch 30/30] train loss : 1.4284 | valid loss : 1.1590 | valid acc : 0.71% | vali

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


[Epoch 30/30] train loss : 1.9182 | valid loss : 1.7977 | valid acc : 0.37% | valid f1 score : 0.2232
[Epoch 30/30] train loss : 1.8654 | valid loss : 1.7722 | valid acc : 0.31% | valid f1 score : 0.1857
[Epoch 30/30] train loss : 1.7887 | valid loss : 1.6706 | valid acc : 0.39% | valid f1 score : 0.3188
[Epoch 30/30] train loss : 1.7553 | valid loss : 1.6108 | valid acc : 0.40% | valid f1 score : 0.3491
[Epoch 30/30] train loss : 1.7304 | valid loss : 1.4989 | valid acc : 0.46% | valid f1 score : 0.4207
[Epoch 30/30] train loss : 1.6408 | valid loss : 1.4066 | valid acc : 0.53% | valid f1 score : 0.4883
[Epoch 30/30] train loss : 1.5517 | valid loss : 1.3321 | valid acc : 0.57% | valid f1 score : 0.5308
[Epoch 30/30] train loss : 1.4741 | valid loss : 1.2721 | valid acc : 0.61% | valid f1 score : 0.5668
[Epoch 30/30] train loss : 1.4248 | valid loss : 1.2141 | valid acc : 0.64% | valid f1 score : 0.5973
[Epoch 30/30] train loss : 1.3518 | valid loss : 1.1630 | valid acc : 0.67% | vali

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


[Epoch 30/30] train loss : 1.9065 | valid loss : 1.9062 | valid acc : 0.21% | valid f1 score : 0.1127
[Epoch 30/30] train loss : 1.8850 | valid loss : 1.8174 | valid acc : 0.29% | valid f1 score : 0.1696
[Epoch 30/30] train loss : 1.9101 | valid loss : 1.7464 | valid acc : 0.30% | valid f1 score : 0.1981
[Epoch 30/30] train loss : 1.8453 | valid loss : 1.6896 | valid acc : 0.32% | valid f1 score : 0.2305
[Epoch 30/30] train loss : 1.7699 | valid loss : 1.6137 | valid acc : 0.39% | valid f1 score : 0.3014
[Epoch 30/30] train loss : 1.7134 | valid loss : 1.5223 | valid acc : 0.46% | valid f1 score : 0.3687
[Epoch 30/30] train loss : 1.6461 | valid loss : 1.4372 | valid acc : 0.52% | valid f1 score : 0.4410
[Epoch 30/30] train loss : 1.5682 | valid loss : 1.3620 | valid acc : 0.57% | valid f1 score : 0.4965
[Epoch 30/30] train loss : 1.5057 | valid loss : 1.3032 | valid acc : 0.60% | valid f1 score : 0.5370
[Epoch 30/30] train loss : 1.4430 | valid loss : 1.2559 | valid acc : 0.63% | vali

In [45]:
test_transforms = A.Compose([
    A.Normalize()
])

test_files = os.listdir("/content/drive/MyDrive/test/test_image")
test_files = sorted(test_files)
test_files = list(map(lambda x: "/".join(["/content/drive/MyDrive/test/test_image", x]), test_files))

test_dataset = TestDataset(file_lists=test_files, transforms=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


answer_logits = []

model = timm.create_model(model_name_, pretrained=True, num_classes=7).to(device=device)

for fold_idx, best_model in enumerate(best_models, 1):
    model.load_state_dict(best_model)
    model.eval()

    fold_logits = []

    with torch.no_grad():
        for iter_idx, test_imgs in enumerate(test_loader, 1):
            test_imgs = test_imgs.to(device)

            test_pred = model(test_imgs)
            fold_logits.extend(test_pred.cpu().tolist())

            print(f"[{fold_idx} fold] inference iteration {iter_idx}/{len(test_loader)}" + " " * 10, end="\r")

    answer_logits.append(fold_logits)

answer_logits = np.mean(answer_logits, axis=0)
answer_value = np.argmax(answer_logits, axis=-1)

i = 0
while True:
    if not os.path.isfile(os.path.join("submissions", f"submission_{i}.csv")):
        submission_path = os.path.join("submissions", f"submission_{i}.csv")
        os.makedirs("submissions", exist_ok=True)
        break
    i += 1

csv_path = '/content/drive/MyDrive/test_answer_sample_.csv'
submission = pd.read_csv(csv_path, index_col=False)
submission["answer value"] = answer_value
submission["answer value"].to_csv(submission_path)
print("\nAll done.")

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name swsl_resnext50_32x4d to current resnext50_32x4d.fb_swsl_ig1b_ft_in1k.
  model = create_fn(


[5 fold] inference iteration 8/8          
All done.
